In [39]:
BASE_URL = "https://www.otodom.pl"
SEARCH_URL = BASE_URL + "/pl/wyniki/sprzedaz/mieszkanie/malopolskie/krakow/krakow/krakow?viewType=listing&page="
ulr = SEARCH_URL + "1"
print(ulr)

https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/malopolskie/krakow/krakow/krakow?viewType=listing&page=1


In [40]:
import requests
from bs4 import BeautifulSoup

def load_page(url: str) -> BeautifulSoup:
    res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup

def get_links_for_search_page(n: int) -> list[str]:
    url = SEARCH_URL + str(n)
    soup = load_page(url)
    links = [BASE_URL + a['href'] for a in soup.find_all('a', attrs={'data-cy': 'listing-item-link'})]
    return links

In [41]:
links = get_links_for_search_page(1)
links[0]

'https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-38m2-balkon-bez-prowizji-ID4ul5O'

In [42]:
def extract_data(url: str, soup: BeautifulSoup):
    slug = url.split('/')[-1] # this is unique identifier which we can use as primary key
    name = soup.find('h1', attrs={'data-cy': 'adPageAdTitle'}).text
    price = soup.find('strong', attrs={'aria-label': 'Cena'}).text
    info = soup.find_all('div', attrs={'class': 'css-1ftqasz'})
    area = info[0].text
    rooms = info[1].text
    
    info_table = soup.find('div', {'class': 'css-t7cajz e16p81cp1'}).parent
    info_table = [element for element in info_table.children if element.name == 'div']
    cols = ['heating', 'floor', 'rent', 'state', 'market', 'ownership', 'available', 'ad_type', 'extra_info']
    data = {col: info_table[idx].text for col, idx in zip(cols, range(0, 18, 2))}

    data.update({'slug': slug, 'price': price, 'name': name, 'area': area, 'rooms': rooms})
    return data

In [43]:
import pandas as pd

def get_one_search_page(n: int) -> pd.DataFrame:
    links = get_links_for_search_page(n)
    df = pd.DataFrame()
    data_list = []
    for link in links:
        print(f"Checking {link}")
        soup = load_page(link)
        data = extract_data(link, soup)
        data_list.append(data)
    df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)
    df.set_index('slug', inplace=True)
    return df

In [44]:
def get_n_pages(n: int, offset:int=0) -> pd.DataFrame:
    df = pd.DataFrame()
    for i in range(offset, n):
        df = pd.concat([df, get_one_search_page(i)], ignore_index=False)
    return df

In [45]:
df = get_n_pages(1)

Checking https://www.otodom.pl/pl/oferta/duzy-ogrodek-120m2-mieszkanie-44-42m2-ID4uJ2T
Checking https://www.otodom.pl/pl/oferta/2-pokoje-na-podgorzu-wielicka-ID4tgDZ
Checking https://www.otodom.pl/pl/oferta/kawalerka-po-generalnym-remoncie-ID4tHWr
Checking https://www.otodom.pl/pl/oferta/3-pokojowe-mieszkanie-57m2-balkon-bezposrednio-ID4t4xf
Checking https://www.otodom.pl/pl/oferta/5-pokojowe-mieszkanie-162m2-taras-ID4uhOa
Checking https://www.otodom.pl/pl/oferta/nowy-2-3-pok-apartament-krowodrza-browar-kleparz-ID4sOpK
Checking https://www.otodom.pl/pl/oferta/ul-glogera-mieszkanie-4-pok-h-36-ID4tLEZ
Checking https://www.otodom.pl/pl/oferta/3-pokojowe-mieszkanie-61m2-taras-bez-prowizji-ID4uEIa
Checking https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-113m2-bez-prowizji-ID4u8DM
Checking https://www.otodom.pl/pl/oferta/tania-kawalerka-w-poblizu-placu-inwalidow-ID4s8IA
Checking https://www.otodom.pl/pl/oferta/przestronne-3m-i-do-wejscia-i-garaz-i-ID4uJqc
Checking https://www.otodom.pl

In [46]:
# Check what we've got - random 5 rows
print(df.shape)
df.sample(5)

(30, 13)


,heating,floor,rent,state,market,ownership,available,ad_type,extra_info,price,name,area,rooms
slug,,,,,,,,,,,,,
sprzedam-apartament-w-kamienicy-po-renowacji-ID4tPx4,Ogrzewanie:miejskie,Piętro:1,Czynsz:650 zł,Stan wykończenia:do zamieszkania,Rynek:wtórny,Forma własności:pełna własność,Dostępne od:brak informacji,Typ ogłoszeniodawcy:prywatny,Informacje dodatkowe:balkon,1 117 000 zł,"Sprzedam, apartament w kamienicy po renowacji",42.58m²,2 pokoje
studio-funkcjonalny-uklad-panoramiczny-widok-ID4rDNE,Ogrzewanie:miejskie,Piętro:5/6,Czynsz:brak informacji,Stan wykończenia:do wykończenia,Rynek:pierwotny,Forma własności:pełna własność,Dostępne od:brak informacji,Typ ogłoszeniodawcy:biuro nieruchomości,Informacje dodatkowe:brak informacji,575 960 zł,studio | funkcjonalny układ | panoramiczny widok,26.18m²,1 pokój
6-pokojowy-apartament-inwestycyjny-i-slowackiego-ID4tJfD,Ogrzewanie:gazowe,Piętro:1/4,Czynsz:1 184 zł,Stan wykończenia:do zamieszkania,Rynek:wtórny,Forma własności:pełna własność,Dostępne od:2024-11-06,Typ ogłoszeniodawcy:biuro nieruchomości,Informacje dodatkowe:balkon piwnica,2 450 000 zł,6- Pokojowy Apartament Inwestycyjny I Słowackiego,163.35m²,6 pokoi
3-pokojowe-mieszkanie-57m2-balkon-bezposrednio-ID4t4xf,Ogrzewanie:brak informacji,Piętro:2/5,Czynsz:brak informacji,Stan wykończenia:do wykończenia,Rynek:pierwotny,Forma własności:pełna własność,Dostępne od:2024-06-06,Typ ogłoszeniodawcy:deweloper,Informacje dodatkowe:balkon garaż/miejsce park...,861 010 zł,3-pokojowe mieszkanie 57m2 + balkon Bezpośrednio,57.21m²,3 pokoje
2poziomowy-apartament-panoramiczny-widok-centrum-ID4rGNW,Ogrzewanie:gazowe,Piętro:4/4,Czynsz:brak informacji,Stan wykończenia:do wykończenia,Rynek:pierwotny,Forma własności:pełna własność,Dostępne od:brak informacji,Typ ogłoszeniodawcy:biuro nieruchomości,Informacje dodatkowe:brak informacji,1 739 475 zł,2poziomowy apartament|panoramiczny widok| centrum,77.31m²,4 pokoje


In [47]:
df.to_csv('otodom.csv')